# How to avoid "SQL injection"?
### using `%s` token

In [ ]:
def add_input(self, data):
    connection = self.connect()
    try:
        q = "INSERT INTO crimes (description) VALUES (%s);"
        with connection.cursor() as cur:
            cur.execute(q)
            connection.commit()
    finally:
        connection.close()

In [ ]:
#dbhelper.py
import pymysql
import ch6_dbconfig

class DBHelper:
    
    def connect(self, database="crimemap"):
        return pymysql.connect(host="54.180.124.182",
                               user=ch6_dbconfig.db_user,
                               passwd=ch6_dbconfig.db_password,
                               db=database)
    
    def get_all_inputs(self):
        connection = self.connect()
        try:
            q = "SELECT description FROM crimes;"
            with connection.cursor() as cur:
                cur.execute(q)
            return cur.fetchall()
        finally:
            connection.close()
        
    def add_input(self, data):
        connection = self.connect()
        try:
            q = "INSERT INTO crimes (description) VALUES('{}')".format(data)
            with connection.cursor() as cur:
                cur.execute(q)
                connection.commit()
        finally:
            connection.close()
    
    def clear_all(self):
        connection = self.connect()
        try:
            q = "DELETE FROM crimes;"
            with connection.cursor() as cur:
                cur.execute(q)
                connection.commit()
        finally:
            connection.close()
            

# How to get user input data?
### submit - /add(save) - /home(return)
- 1) click submit
- 2) **request** data in `userinput` by POST method
- 3) .py `/add` receive data 
- 4) .py `add_input()` **save** data in DB
- 5) .py return `home()`
- 6) .py `get_all_inputs() `**get** data
- 7) .py return data having .html display data


In [ ]:
#mod1_ch6.py
from dbhelper import DBHelper
from flask import Flask
from flask import render_template
from flask import request

app = Flask(__name__)
DB = DBHelper()

@app.route("/")
def home():
    try:
        data = DB.get_all_inputs()
    except Exception as e:
        print(e)
        data = None
    return render_template("home_ch6.html", data=data)

@app.route("/add", methods=["POST"])
def add():
    try:
        data = request.form.get("userinput")
        DB.add_input(data)
    except Exception as e:
        print(e)
    return home()

@app.route("/clear")
def clear():
    try:
        DB.clear_all()
    except Exception as e:
        print(e)
    return home()

if __name__ == '__main__':
    app.run(port=5000, debug=True)

In [ ]:
# sql_setup
import pymysql
import ch6_dbconfig
connection = pymysql.connect(host="54.180.124.182",
                            user=ch6_dbconfig.db_user,
                            passwd=ch6_dbconfig.db_password)
try:
    with connection.cursor() as cur:
        query = "CREATE DATABASE IF NOT EXISTS crimemap"
        cur.execute(query)
        query="""
        CREATE TABLE IF NOT EXISTS crimemap.crimes (
        id int NOT NULL AUTO_INCREMENT,
        latitude FLOAT(10,6),
        longitude FLOAT(10,6),
        date DATETIME,
        category VARCHAR(50),
        description VARCHAR(1000),
        updated_at TIMESTAMP,
        PRIMARY KEY (id)
        )
        """
        cur.execute(query)
        connection.commit()
finally:
    connection.close()
